In [1]:
import pandas as pd
import numpy as np

In [2]:
df2 = pd.read_csv('../scraped-data-2024/cluster2-html.csv')
df2 = pd.concat([df2, pd.read_csv('../scraped-data-2024/cluster2_5-pdf.csv')])

In [3]:
df2.replace('\xa0', np.nan, inplace=True)
df2.dropna(subset=["Pontuação Regata"], inplace=True)
df2['Punição'] = 0
df2['Descarte'] = 0

In [4]:
import re
import unidecode

def process_string(s):
    # Remove espaços em branco extras
    s = str(s).strip()
    
    numbers = re.findall(r'\d+(?:\.\d+)?', s)
    
    has_parentheses = 1 if '(' in s or ')' in s else 0
    
    letters = ''.join(re.findall(r'[a-zA-Z]', s))
    if letters == '':
        letters = 0
    return numbers, has_parentheses, letters

In [5]:
df2['Punição'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[2])
df2['Descarte'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[1])
df2['Pontuação Regata'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[0][0])
df2["Nome Competidor"] = df2["Nome Competidor"] .apply(lambda x: unidecode.unidecode(x).upper())

In [6]:
df2.to_csv('../cleaned_data_2024/cluster_2.csv', index=False)

In [7]:
df3 = pd.read_csv('../scraped-data-2024/cluster3.csv')

In [8]:
df3.head()

,Id Resultado,Id Competidor,Nome Competidor,Id Competição,Classe Vela,Pontuação Regata,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição,Regata
0,NaN,NaN,THAMSOONTORN/KHEMKAEW,NaN,470,18.0,G,49,NaN,210.0,181.0,World Championship 2023,Race 9 Silver
1,NaN,NaN,GIANNOULIS/GIANNOULI,NaN,470,32.0,G,58,NaN,258.0,226.0,World Championship 2023,Race 9 Silver
2,NaN,NaN,JERWOOD/NICHOLAS,NaN,470,10.0,G,36,NaN,135.0,102.0,World Championship 2023,Race 9 Silver
3,NaN,NaN,SPANAKI/SPANAKIS,NaN,470,2.0,G,48,NaN,211.0,178.0,World Championship 2023,Race 9 Silver
4,NaN,NaN,BRISTOW/TAYLOR,NaN,470,5.0,G,38,NaN,137.0,116.0,World Championship 2023,Race 9 Silver
